In [1]:
!pip install datasets accelerate hf-transfer

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
# Load SST-2 dataset
dataset = load_dataset("glue", "stsb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

stsb/train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

stsb/validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

stsb/test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load tokenizer and models
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-large")
teacher_cls = AutoModelForSequenceClassification.from_pretrained("answerdotai/ModernBERT-large",  num_labels=1, problem_type="regression")
student_cls = AutoModelForSequenceClassification.from_pretrained("kartikeya-pandey/MiniModernBERT-Pretrained", num_labels=1,problem_type="regression")

teacher_cls = teacher_cls.to("cuda")
student_cls = student_cls.to("cuda")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/505M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at kartikeya-pandey/MiniModernBERT-Pretrained and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Preprocessing for STSB
def preprocess(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding="max_length", max_length=128)

In [10]:
tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [11]:
import torch
import torch.nn.functional as F
from torch.nn import KLDivLoss, MSELoss

def compute_distill_loss(student_logits, teacher_logits, labels, alpha=0.6, temperature=1.0):
    # KL divergence between teacher and student (softened logits)
    kl_loss = KLDivLoss(reduction="batchmean")(
        F.log_softmax(student_logits / temperature, dim=-1),
        F.softmax(teacher_logits / temperature, dim=-1)
    ) * (temperature ** 2)

    # MSE for regression
    mse_loss = MSELoss()(student_logits.view(-1), labels.view(-1))

    return alpha * kl_loss + (1 - alpha) * mse_loss

In [12]:
# Custom compute_loss for Trainer
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs["labels"].float()
    with torch.no_grad():
        teacher_outputs = teacher_cls(**inputs)

    student_outputs = model(**inputs)
    loss = compute_distill_loss(
        student_outputs.logits,
        teacher_outputs.logits,
        labels
    )

    return (loss, student_outputs) if return_outputs else loss

In [13]:
student_cls.compute_loss = compute_loss.__get__(student_cls)

In [14]:
from transformers import TrainingArguments, Trainer

# Training arguments
args = TrainingArguments(
    output_dir="./MiniModernBERT-glue-stsb",
    per_device_train_batch_size=8,
    num_train_epochs=4,
    learning_rate=3e-5,
    fp16=True,
    logging_steps=200,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=student_cls,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer
)

# Fine-tune the student model
trainer.train()

/tmp/ipython-input-1704181432.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.
W0910 08:19:59.432000 171 torch/_inductor/utils.py:1436] [1/0_1] Not enough SMs to use max_autotune_gemm mode


Step,Training Loss
200,1.661800
400,0.996900
600,0.879600
800,0.624100
1000,0.407600
1200,0.413200
1400,0.393900
1600,0.205300
1800,0.171200
2000,0.165200


TrainOutput(global_step=2876, training_loss=0.4358826960244, metrics={'train_runtime': 460.1939, 'train_samples_per_second': 49.97, 'train_steps_per_second': 6.25, 'total_flos': 1317422734605312.0, 'train_loss': 0.4358826960244, 'epoch': 4.0})

In [15]:
# Remove the custom loss before normal evaluation
del student_cls.compute_loss

In [16]:
 trainer.evaluate()

{'eval_loss': 0.6331796646118164,
 'eval_runtime': 5.6244,
 'eval_samples_per_second': 266.694,
 'eval_steps_per_second': 33.426,
 'epoch': 4.0}

In [17]:
# Install evaluation library
!pip install evaluate

In [18]:
import evaluate

pearson = evaluate.load("pearsonr")
spearman = evaluate.load("spearmanr")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    return {
        "pearson": pearson.compute(predictions=predictions, references=labels)["pearsonr"],
        "spearman": spearman.compute(predictions=predictions, references=labels)["spearmanr"],
    }


In [19]:
from transformers import Trainer

trainer = Trainer(
    model=student_cls,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2932533570.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6331796646118164, 'eval_model_preparation_time': 0.002, 'eval_pearson': 0.8480588728588168, 'eval_spearman': 0.8461149480037623, 'eval_runtime': 3.7769, 'eval_samples_per_second': 397.155, 'eval_steps_per_second': 49.777}


In [22]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `Kartikeya Pandey` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root

In [23]:
trainer.push_to_hub("kartikeya-pandey/MiniModernBERT-stsb")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/505M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.78k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kartikeyapandey20/MiniModernBERT-glue-stsb/commit/12942a0398852cedb1a50a25ed727abeb3ee8528', commit_message='kartikeya-pandey/MiniModernBERT-stsb', commit_description='', oid='12942a0398852cedb1a50a25ed727abeb3ee8528', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kartikeyapandey20/MiniModernBERT-glue-stsb', endpoint='https://huggingface.co', repo_type='model', repo_id='kartikeyapandey20/MiniModernBERT-glue-stsb'), pr_revision=None, pr_num=None)